In [1]:
import pandas as pd



In [2]:
from nsepy import get_history as gh
import datetime as dt



In [3]:
import pandas as pd
import datetime as dt
import kagglehub
import os

# Load the dataset
path = kagglehub.dataset_download("vijayvvenkitesh/microsoft-stock-time-series-analysis")
df = pd.read_csv(os.path.join(path, "Microsoft_Stock.csv"))
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Auto-select 150 days around the midpoint
mid_date = df['Date'].min() + (df['Date'].max() - df['Date'].min()) / 2
start = mid_date - pd.Timedelta(days=75)
end = mid_date + pd.Timedelta(days=75)

stk_data = df[(df['Date'] >= start) & (df['Date'] <= end)]
stk_data.set_index('Date', inplace=True)

print(f"Selected 150-day range: {start.date()} → {end.date()}")
print(f"Total rows: {len(stk_data)}")
print(stk_data.head())

Selected 150-day range: 2018-01-16 → 2018-06-15
Total rows: 105
                      Open   High    Low  Close    Volume
Date                                                     
2018-01-16 16:00:00  90.10  90.79  88.01  88.35  36599736
2018-01-17 16:00:00  89.08  90.28  88.75  90.14  25621164
2018-01-18 16:00:00  89.80  90.67  89.66  90.10  24159683
2018-01-19 16:00:00  90.14  90.61  89.66  90.00  36875013
2018-01-22 16:00:00  90.00  91.62  89.74  91.61  23601555


In [4]:
stk_data=stk_data[["Open","High","Low","Close"]]

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (105, 4)


In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [7]:
data1

,Open,High,Low,Close
0,0.232416,0.135174,0.231323,0.191076
1,0.170031,0.098110,0.272274,0.293478
2,0.214067,0.126453,0.322634,0.291190
3,0.234862,0.122093,0.322634,0.285469
4,0.226300,0.195494,0.327061,0.377574
...,...,...,...,...
100,0.904587,0.946221,0.924737,0.950801
101,0.921713,0.920058,0.931931,0.917620
102,0.905199,0.909884,0.936359,0.932494
103,0.943119,0.950581,0.925844,0.906178


In [8]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)


84
X_train length: (84, 4)
X_test length: (21, 4)
y_train length: (84, 4)
y_test length: (21, 4)


In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [11]:
performance


{'Model': [], 'RMSE': [], 'MaPe': [], 'Lag': [], 'Test': []}

In [12]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [13]:
listt=["Close","High","Open","Low"]

In [14]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -23.551020252975444
BIC:  -22.937669373952726

Order = 2
AIC:  -23.788502094104096
BIC:  -22.67610779960667

Order = 3
AIC:  -23.785446700982074
BIC:  -22.166373932784552

Order = 4
AIC:  -23.81650924677218
BIC:  -21.682930589264092

Order = 5
AIC:  -23.64153487436349
BIC:  -20.98542440217809

Order = 6
AIC:  -23.755897449872176
BIC:  -20.569024383616803

Order = 7
AIC:  -23.754865853723388
BIC:  -20.02878802404159

Order = 8
AIC:  -23.675758550155063
BIC:  -19.401815671795088

Order = 9
AIC:  -23.75163597159046
BIC:  -18.920942730913048

Order = 10
AIC:  -23.920474900947376
BIC:  -18.52391386243815



In [15]:
data1

,Open,High,Low,Close
0,0.232416,0.135174,0.231323,0.191076
1,0.170031,0.098110,0.272274,0.293478
2,0.214067,0.126453,0.322634,0.291190
3,0.234862,0.122093,0.322634,0.285469
4,0.226300,0.195494,0.327061,0.377574
...,...,...,...,...
100,0.904587,0.946221,0.924737,0.950801
101,0.921713,0.920058,0.931931,0.917620
102,0.905199,0.909884,0.936359,0.932494
103,0.943119,0.950581,0.925844,0.906178


In [16]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.2843,12,28


In [17]:
result

In [18]:
pred

array([[0.50457532, 0.43278463, 0.53062082, 0.64594939],
       [0.582889  , 0.48310745, 0.41107467, 0.53071268],
       [0.66151936, 0.53903485, 0.57544848, 0.71572105],
       [0.45760216, 0.52012332, 0.66866389, 0.68832066],
       [0.66460677, 0.48989632, 0.36036977, 0.50698369],
       [0.72092103, 0.64060491, 0.76089959, 0.79809676],
       [0.47229735, 0.62307371, 0.73681285, 0.61618466],
       [0.70131278, 0.51209564, 0.49364668, 0.53062334],
       [0.61278262, 0.61796952, 0.80957079, 0.7345091 ],
       [0.57854809, 0.54299866, 0.57801705, 0.51099781],
       [0.79008691, 0.59722361, 0.63766403, 0.67726185],
       [0.46260673, 0.67010633, 0.84728134, 0.63699837],
       [0.6558259 , 0.47824862, 0.37312112, 0.40227379],
       [0.59380647, 0.59012622, 0.70286259, 0.67985807],
       [0.30091134, 0.54197659, 0.61785002, 0.4386761 ],
       [0.65832334, 0.40223656, 0.23003926, 0.34043611],
       [0.38542836, 0.51840854, 0.68779283, 0.64527866],
       [0.38163402, 0.39177896,